In [16]:
# importing necessary libraries

import pickle
import numpy as np
import matplotlib.pyplot as plt
import cv2
from shapely.geometry import Polygon
import networkx as nx
import torch
import torch.nn.functional as F
from skimage import transform
import matplotlib.image as mpimg
from torch.utils.data import DataLoader
from floortrans.models import get_model
from floortrans.loaders import FloorplanSVG, DictToTensor, Compose, RotateNTurns
from floortrans.plotting import segmentation_plot, polygons_to_image, draw_junction_from_dict, discrete_cmap
#discrete_cmap()
from floortrans.post_prosessing import split_prediction, get_polygons, split_validation
from mpl_toolkits.axes_grid1 import AxesGrid

In [17]:
rot = RotateNTurns() #

room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")

data_folder = 'data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)

# Setup Model
model = get_model('hg_furukawa_original', 51)
n_classes = 44
split = [21, 12, 11]

In [18]:
def isolate_class(rooms, CLASS: int):
    template = np.zeros_like(rooms)
    rows, cols = np.where(rooms == CLASS)
    template[rows, cols] = 1
    return template
    
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]

In [19]:
#input: a floorplan in numpy, a list of room types in ints
#output: room_contours: a dictionary where each key is a roomtype in int, and the value is a list of the contours where the floorplan has that room
#room_contours[12] : the list of contours where the roomtype is a door
#nodes: a dictionary where each key is a roomtype in int and the value is  like a list of array([522.5, 219. ])
#this is a list of nd array
def vis_nodes(img, significant_nodes):
    # signficant nodes exclude rooms we don't care about
    nodes, room_contours= {},{}
    
    for c in significant_nodes: # c represents the rooms that we care about, in integer representation of rooms_list
        nodes[c], room_contours[c]=[], []
        t = isolate_class(img, c) # this returns np representation of the floorplan, with the pixels being 1 where it is a specific room, 0 elsewhere
        #CHANGED BELOW TO CHANGE_APPROX_SIMPLE
        contours, _ = cv2.findContours(t.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE) #retr_external means ignore internal gaps in images. if we have donut it becomes a circle. chain_approx_none means that all boundary points are stored. chain_approx_simple removes redundant points
        
        for s in contours:
            room_contours[c].append(s) # dictionary is {room_type_as_int:[contour1, contour2...]}
            nodes[c].append(np.squeeze(np.array(s), 1).mean(0)) # dictionary is {room_type_as_int: [(center in horizontal of contour, center in vertical of contour), (x,y for contour 2...)]}

    return(room_contours, room_contours[12], nodes) # room contours, door contours

each node attribute has type and area
pass in contour,polygon and get area

In [20]:
# test is the modified dataset from when we defined some undefined rooms using the count of icons using a classification model

# change this line for each dataset. currently creating embeddings from test
with open("my_data/test_modified_1.pkl", 'rb') as f:
    test=pickle.load(f)

In [21]:
embeddings,Y=None, None # return values

In [23]:
for index, floorplan in test.items():
    if 11 not in set(floorplan.flatten()): # 11 represents undefined rooms. we don't care about floorplans that contain this
        icons=normal_set[index]['label'][1].numpy() #np representations of icons on the floorplan
        row, column=np.where(icons==2) #finding doors on this floorplan
        floorplan[row, column]=12 #modifying the floorplan to include doors as a room
        rooms, doors, nodes=vis_nodes(floorplan, good)
        
    
    

{3: [array([522.5, 219. ])],
 4: [array([478.    , 462.4375])],
 5: [array([111.5, 379. ]),
  array([394.5, 195.5]),
  array([254.5, 157. ]),
  array([111.5, 178. ])],
 6: [array([111.5, 291. ]),
  array([683.5, 282.5]),
  array([758. , 270.5]),
  array([617.5, 254.5])],
 7: [array([217., 400.]),
  array([236.57142857, 292.14285714]),
  array([686.9, 206. ])],
 9: [array([300., 252.])],
 10: [],
 12: [array([205. , 435.5]),
  array([212.5, 368. ]),
  array([187.5, 368. ]),
  array([688. , 361.5]),
  array([171., 347.]),
  array([726., 302.]),
  array([356.5, 290.5]),
  array([171. , 294.5]),
  array([337.5, 251.5]),
  array([171., 235.]),
  array([746. , 218.5]),
  array([623. , 218.5]),
  array([215.5, 213.5]),
  array([793., 187.]),
  array([594., 187.]),
  array([573., 139.])]}

In [27]:
type(nodes[3][0])

numpy.ndarray